In [1]:
%matplotlib widget

Trying to replicate the following research piece:
https://www.federalreserve.gov/econres/notes/feds-notes/index-of-common-inflation-expectations-20200902.htm
Essentially to create an indicator of inflation expectations that reads through the noise of various forward looking metrics.

In [2]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
# from fredapi import Fred as FredApi
# fred = FredApi(api_key='0dfcbb403eea7fee006f01e935be9216')

#### Survey of Professional Forecasters data
Probably the easiest solution is to download data in xlsx format from the university of Michigan data area located at the below and subpages:

- https://www.philadelphiafed.org/surveys-and-data/real-time-data-research/survey-of-professional-forecasters
- https://www.philadelphiafed.org/surveys-and-data/real-time-data-research/inflation-forecasts

Save the files in this folder with appropriate names.
Once you've got those, you have 7 of the 21 series proposed by the article as they use:

* CPI: 1y ahead, 6-10y ahead, next 10y
* PCE: 1y ahead, 6-10y ahead, next 10y
* Core PCE 1y ahead

There is also another 10y ahead from other sources... so why not!
Notably, there may be differences in what they consider as "6-10y". Maybe they consider the implied 5y5y forward that is also available (and calculated from the 5y and the 10y clearly). 

In [3]:
# the file below appears to be having an issue...
# see the following bug report for openpyxl
# https://foss.heptapod.net/openpyxl/openpyxl/-/issues/1659
# solution is to use "mock"

import mock
from openpyxl.reader import excel

with mock.patch.object(excel.ExcelReader, 'read_properties', lambda self: None):
    #sheet = pd.read_excel("spf_meanLevel.xlsx", dtype={'YEAR': str, 'QUARTER': str})
    spf_base = pd.read_excel("spf_meanLevel.xlsx", sheet_name=["CPI","PCE","CPI5YR","CPI10","PCE5YR","PCE10"], 
                             header=0, dtype={"YEAR":int, "QUARTER": int});

#spf_base = pd.read_excel("spf_meanLevel.xlsx", sheet_name=["CPI","PCE","CPI5YR","CPI10","PCE5YR","PCE10"], header=0);

/opt/conda/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""C

In [4]:
pce = spf_base["PCE"]
cpi = spf_base["CPI"]
cpi_5y = spf_base["CPI5YR"]
cpi_10y = spf_base["CPI10"]
pce_5y = spf_base["PCE5YR"]
pce_10y = spf_base["PCE10"]

In [5]:
cpi.tail()

,YEAR,QUARTER,CPI1,CPI2,CPI3,CPI4,CPI5,CPI6,CPIA,CPIB,CPIC
216,2022,4,5.6983,5.2316,4.4592,3.5880,3.4360,3.1985,7.6551,3.7111,2.8363
217,2023,1,3.1018,3.3439,3.3363,3.0288,2.8946,2.6314,3.1519,2.5261,2.3614
218,2023,2,3.8018,3.4737,3.3007,3.0205,2.7712,2.5902,3.4153,2.5996,2.3408
219,2023,3,2.7012,3.0672,2.9960,2.6415,2.5593,2.5843,3.1418,2.5842,2.3732
220,2023,4,3.5984,3.1613,2.7176,2.5583,2.5365,2.4694,3.3480,2.5858,2.3469


In [6]:
# calculate the 1y ahead forecasts as indicated by the paper PDF in the folder
def one_year_forecast(df, lbl):
    df2 = pd.DataFrame(index=df.index, columns=["YEAR","QUARTER",lbl+"1YR"])
    df2[lbl+"1YR"] = 100*(((1+df[lbl+"3"]/100)*(1+df[lbl+"4"]/100)*(1+df[lbl+"5"]/100)*(1+df[lbl+"6"]/100))**0.25 - 1)
    df2[["YEAR","QUARTER"]] = df[["YEAR","QUARTER"]]
    return df2

cpi_1y = one_year_forecast(cpi, "CPI")
pce_1y = one_year_forecast(pce, "PCE")

In [7]:
# now create an index based on the years and quarters by working on a separate temporary dataframe
df_tmp = cpi[["YEAR","QUARTER"]].copy()
df_tmp["Year"] = df_tmp["YEAR"].astype(int)
df_tmp["Quarter"] = df_tmp["QUARTER"].astype(int)
df_tmp["Month"] = df_tmp["Quarter"]*3
df_tmp["Day"] = 1
df_tmp["Date"] = pd.to_datetime(df_tmp[["Year","Month","Day"]]) + pd.offsets.QuarterEnd(0)
df_tmp.set_index("Date",inplace=True)
# df_tmp.tail()

In [8]:
# now create the final dataframe
# and add the implied 5y5y fwd calculations based on the 5y and the 10y numbers (1+5y)^5 * (1+5y5y)^5 == (1+10y)^10
df_idx = pd.date_range(start=df_tmp.index.min(), end=df_tmp.index.max(), freq='Q')
df = pd.DataFrame(index=df_idx)
df["spf_cpi1y"] = cpi_1y["CPI1YR"].to_numpy()
df["spf_cpi5y"] = cpi_5y["CPI5YR"].to_numpy()
df["spf_cpi10y"] = cpi_10y["CPI10"].to_numpy()
df["spf_cpi5y5y"] =( ((1 + df["spf_cpi10y"]/100)**10 / (1 + df["spf_cpi5y"]/100)**5)**0.2 - 1)*100
df["spf_pce1y"] = pce_1y["PCE1YR"].to_numpy()
df["spf_pce5y"] = pce_5y["PCE5YR"].to_numpy()
df["spf_pce10y"] = pce_10y["PCE10"].to_numpy()
df["spf_pce5y5y"] =( ((1 + df["spf_pce10y"]/100)**10 / (1 + df["spf_pce5y"]/100)**5)**0.2 - 1)*100
df.tail()

,spf_cpi1y,spf_cpi5y,spf_cpi10y,spf_cpi5y5y,spf_pce1y,spf_pce5y,spf_pce10y,spf_pce5y5y
2022-12-31,3.669335,3.8948,3.0650,2.241828,3.250721,3.2761,2.6134,1.954952
2023-03-31,2.972462,2.6546,2.4402,2.226248,2.670856,2.3679,2.2280,2.088291
2023-06-30,2.920303,2.6058,2.4060,2.206589,2.792950,2.4137,2.1890,1.964793
2023-09-30,2.695124,2.6826,2.4169,2.151888,2.512892,2.4819,2.2521,2.022815
2023-12-31,2.570410,2.6248,2.4318,2.239163,2.355114,2.4232,2.2300,2.037164


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 221 entries, 1968-12-31 to 2023-12-31
Freq: Q-DEC
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   spf_cpi1y    170 non-null    float64
 1   spf_cpi5y    74 non-null     float64
 2   spf_cpi10y   129 non-null    float64
 3   spf_cpi5y5y  74 non-null     float64
 4   spf_pce1y    68 non-null     float64
 5   spf_pce5y    68 non-null     float64
 6   spf_pce10y   68 non-null     float64
 7   spf_pce5y5y  68 non-null     float64
dtypes: float64(8)
memory usage: 15.5 KB


In [10]:
df.to_pickle('spf_dataset.pd.pkl')